In [ ]:
# Topic model- тематическое моделирование-анализ больших объемов текстовых данных путем кластеризации документов в группы. 
# Текстовые данные не имеют прикрепленных к ним меток.Topic model пытается сгруппировать документы в кластеры на основе сходных 
# характеристик. Используются два подхода: Латентное распределение Дирихле и Неотрицательная матричная факторизация. 

In [ ]:
# Латентное распределение Дирихле (LDA)- основывается на двух общих предположениях:
# Документы с похожими словами обычно имеют одну и ту же тему, т.е. документы-это распределения вероятностей по скрытым темам
# Документы, в которых группы слов часто встречаются вместе, обычно имеют одну и ту же тему, т.е.темы-это распределения 
# вероятностей по словам

In [1]:
import numpy as np
import pandas as pd

reviews_datasets = pd.read_csv('D:/Python_Projects/Recomendation_Systems/reviews.csv')
reviews_datasets = reviews_datasets.head(5000) # фильтруем первые 5000 строк
reviews_datasets.dropna() # удаляем нулевые значения из набора данных.

reviews_datasets.head() # печатаем первые пять строк набора данных

,Id,ProfileName,ratings,Time,Summary,review
0,1,delmartian,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,dll pa,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,"Natalia Corres ""Natalia Corres""",4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,Karl,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,"Michael D. Bigham ""M. Wassir""",5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
reviews_datasets['review'][500] # применим LDA к столбцу review, посмотрим номер 500

In [2]:
#Прежде чем применить LDA, создадим словарь всех слов в наших данных. 

from sklearn.feature_extraction.text import CountVectorizer #создание матрицы терминов документа
count_vect = CountVectorizer(max_df=0.8, min_df=3, stop_words='english') #включаем только те слова, которые появляются 
#менее чем в 80% документа и появляются как минимум в 3 документах и удаляем все стоп-слова
doc_term_matrix = count_vect.fit_transform(reviews_datasets['review'].values.astype('U'))
doc_term_matrix

<5000x7013 sparse matrix of type '<class 'numpy.int64'>'
	with 139322 stored elements in Compressed Sparse Row format>

In [ ]:
# Каждый из 5000 документов представлен в виде 7013-мерного вектора, что означает, что словарь содержит 7013 слов.

In [29]:
# Используем LDA для создания тем вместе с распределением вероятностей для каждого слова в словаре для каждой темы.
from sklearn.decomposition import LatentDirichletAllocation
LDA = LatentDirichletAllocation(n_components=8, random_state=30) #n_components- количество тем, на которые хотим разделить текст
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(n_components=8, random_state=30)

In [30]:
# Cлучайным образом извлечем 8 слов из словаря
import random

for i in range(8):
    random_id = random.randint(0,len(count_vect.get_feature_names_out()))
    print(count_vect.get_feature_names_out()[random_id])

concoction
site
ramen
women
lunch
falling
mate
useful


In [31]:
first_topic = LDA.components_[0]
print(first_topic)

[5.09340743 0.29623534 0.12524734 ... 0.12500005 0.1251496  0.12511594]


In [32]:
# Найдем 8 слов с наибольшей вероятностью для первой темы.
top_topic_words = first_topic.argsort()[-8:]
print(first_topic)

[5.09340743 0.29623534 0.12524734 ... 0.12500005 0.1251496  0.12511594]


In [33]:
# Получившиеся индексы можно использовать для извлечения значения слов из объекта count_vector
for i in top_topic_words:
    print(count_vect.get_feature_names_out()[i])

dumplings
flipside
burritos
gamble
drank
breezy
exercising
bakery


In [34]:
# Слова показывают, что первая тема может быть о картошке фри

In [39]:
# Напечатаем 8 слов с наибольшей вероятностью для всех 8 тем:
for i,topic in enumerate(LDA.components_):
    print(f'Top 8 words for topic #{i}:')
    print([count_vect.get_feature_names_out()[i] for i in topic.argsort()[-8:]])
    print('\n')

Top 8 words for topic #0:
['dumplings', 'flipside', 'burritos', 'gamble', 'drank', 'breezy', 'exercising', 'bakery']


Top 8 words for topic #1:
['akg', 'ideas', 'gnc', 'drank', 'leery', 'gadget', 'dumplings', 'bakery']


Top 8 words for topic #2:
['kind', 'gimbal', 'experienced', 'relative', 'killing', 'kills', 'fluctuates', 'killed']


Top 8 words for topic #3:
['inch', 'bakery', 'leery', 'honey', 'dumplings', 'consistency', 'consider', 'dicing']


Top 8 words for topic #4:
['reduces', 'desire', 'drank', 'pleasing', 'gamble', 'pockets', 'bakery', 'bust']


Top 8 words for topic #5:
['gamble', 'drank', 'inputs', 'diluted', 'leery', 'doorstep', 'head', 'bakery']


Top 8 words for topic #6:
['antioxidants', 'gnc', 'desire', 'known', 'dumplings', 'mild', 'bakery', 'breed']


Top 8 words for topic #7:
['book', 'gobbles', 'consistency', 'adding', 'honey', 'preserved', 'consider', 'dicing']




In [40]:
topic_values = LDA.transform(doc_term_matrix) # добавим столбец в исходный фрейм данных, в котором будет храниться 
#тема для текста
topic_values.shape

(5000, 8)

In [41]:
#(5000, 8) означает, что каждый документ имеет 5 столбцов, где каждый столбец соответствует значению 
# вероятности конкретной темы. 

In [42]:
# Добавляем новый столбец для темы во фрейме данных и присваиваем значение темы каждой строке в столбце:
reviews_datasets['Topic'] = topic_values.argmax(axis=1)
reviews_datasets.head()

,Id,ProfileName,ratings,Time,Summary,review,Topic
0,1,delmartian,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,3
1,2,dll pa,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,3
2,3,"Natalia Corres ""Natalia Corres""",4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,5
3,4,Karl,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,5
4,5,"Michael D. Bigham ""M. Wassir""",5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,1


In [43]:
# Неотрицательная матричная факторизация (NMF)-выполняет кластеризацию, а также уменьшение размерности, может быть использована
# в сочетании со схемой TF-IDF для выполнения тематического моделирования.

In [44]:
import pandas as pd
import numpy as np
reviews_datasets = pd.read_csv('D:/Python_Projects/Recomendation_Systems/reviews.csv')
reviews_datasets = reviews_datasets.head(5000)
reviews_datasets.dropna()
reviews_datasets.head()

,Id,ProfileName,ratings,Time,Summary,review
0,1,delmartian,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,dll pa,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,"Natalia Corres ""Natalia Corres""",4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,Karl,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,"Michael D. Bigham ""M. Wassir""",5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [45]:
# Cоздадим матрицу терминов документа с помощью TF IDF.
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=3, stop_words='english')
doc_term_matrix = tfidf_vect.fit_transform(reviews_datasets['review'].values.astype('U'))

In [46]:
# Создаем матрицу вероятностей, которая содержит вероятности всех слов в словаре для всех тем
from sklearn.decomposition import NMF

nmf = NMF(n_components=8, random_state=30)
nmf.fit(doc_term_matrix )

NMF(n_components=8, random_state=30)

In [47]:
# Cлучайным образом получим 8 слов из словаря

import random

for i in range(8):
    random_id = random.randint(0,len(tfidf_vect.get_feature_names_out()))
    print(tfidf_vect.get_feature_names_out()[random_id])

caribou
sauce
crab
dimensional
burst
snacks
female
inhaled


In [48]:
# Получим вектор вероятности слов для первой темы и получим индексы 8 слов с наибольшими вероятностями

first_topic = nmf.components_[0]
top_topic_words = first_topic.argsort()[-8:]

In [49]:
# Эти индексы могут быть переданы объекту tf idf_vector для извлечения фактических слов
for i in top_topic_words:
    print(tfidf_vect.get_feature_names_out()[i])

price
taste
love
just
like
great
good
product


In [50]:
#Напечатаем 8 слов с наибольшей вероятностью для каждой из тем

for i,topic in enumerate(nmf.components_):
    print(f'Top 8 words for topic #{i}:')
    print([tfidf_vect.get_feature_names_out()[i] for i in topic.argsort()[-8:]])
    print('\n')

Top 8 words for topic #0:
['price', 'taste', 'love', 'just', 'like', 'great', 'good', 'product']


Top 8 words for topic #1:
['oil', 'amazon', 'use', 'ingredients', 'sugar', 'fat', 'water', 'br']


Top 8 words for topic #2:
['chip', 'flavor', 'vinegar', 'bag', 'kettle', 'potato', 'salt', 'chips']


Top 8 words for topic #3:
['make', 'waffles', 'pancake', 'bisquick', 'pancakes', 'free', 'mix', 'gluten']


Top 8 words for topic #4:
['black', 'like', 'water', 'teas', 'drink', 'iced', 'green', 'tea']


Top 8 words for topic #5:
['smooth', 'melitta', 'like', 'taste', 'strong', 'decaf', 'cup', 'coffee']


Top 8 words for topic #6:
['milk', 'tried', 'keurig', 'cups', 'cup', 'cocoa', 'hot', 'chocolate']


Top 8 words for topic #7:
['year', 'organic', 'old', 'loves', 'newman', 'dogs', 'dog', 'food']




In [51]:
# Добавляем темы в набор данных и отображаем первые пять строк:
topic_values = nmf.transform(doc_term_matrix)
reviews_datasets['Topic'] = topic_values.argmax(axis=1)
reviews_datasets.head()

,Id,ProfileName,ratings,Time,Summary,review,Topic
0,1,delmartian,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,7
1,2,dll pa,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0
2,3,"Natalia Corres ""Natalia Corres""",4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,0
3,4,Karl,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,0
4,5,"Michael D. Bigham ""M. Wassir""",5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,0


In [ ]:
# Каждому обзору была назначена тема, которая была сгенерирована с помощью метода NMF.